This notebook is inspired from the textbook:

[Jake VanderPlas: Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/index.html)

In [ ]:
import pandas as pd
import numpy as np

## Agenda

- How to read data with different encodings
- How to find missing values in dataframes
- Dealing with missing values
- Combining dataframes
- Groupby in pandas

## Reading Data - Encodings

[Download Data](https://www.kaggle.com/kwullum/fatal-police-shootings-in-the-us?select=PoliceKillingsUS.csv)

[Washington Post Repo](https://github.com/washingtonpost/data-police-shootings)

In [ ]:
# df = pd.read_csv('data/datasets_2647_4395_PoliceKillingsUS.csv')

In [ ]:
before = "This is the euro symbol: €"

In [ ]:
after = before.encode()

after

In [ ]:
# after.decode('ascii')

In [ ]:
ascii_after = before.encode('ascii', errors= 'replace')

In [ ]:
ascii_after.decode('ascii')

In [ ]:
## Solution to the encoding-decoding mess chardet library

In [ ]:
import chardet
# look at the first ten thousand bytes to guess the character encoding
with open("data/datasets_2647_4395_PoliceKillingsUS.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(2000))

# check what the character encoding might be
print(result)

- [Kaggle - Dealing with Char Encodings](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings)

- [Python Standard Encodings](https://docs.python.org/3/library/codecs.html#standard-encodings)

In [ ]:
df = pd.read_csv('data/datasets_2647_4395_PoliceKillingsUS.csv', encoding='Windows-1252')

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
## Detecting Null Values

In [ ]:
df.isna().sum()

[Difference between `isna` and  `isnull`](https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas)

## Dealing with Missing Values

### Droping Rows with NaN values

In [ ]:
new_df = df.dropna(axis ='rows')
new_df.isna().sum()

In [ ]:
new_df = df.dropna(axis ='rows', how = 'all')
new_df.isna().sum()

In [ ]:
new_df = df.dropna(axis ='rows', thresh= 13)
new_df.isna().sum()

In [ ]:
df.dropna(subset = ['race'], inplace = True)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

### Filling Null Values

In [ ]:
mean_age = int(df.age.mean())

df.age.fillna(value = mean_age, inplace= True)

In [ ]:
df.age.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.city.value_counts()

## `Apply`, `Map` and `ApplyMap`

[Dataset](https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-cities-and-towns.html)

In [ ]:
census = pd.read_excel('data/SUB-IP-EST2019-ANNRNK.xlsx')

In [ ]:
census.head();

In [ ]:
census = pd.read_excel('data/SUB-IP-EST2019-ANNRNK.xlsx', skiprows= 3, skipfooter= 5, )

In [ ]:
census.head(2);

In [ ]:
census.columns.tolist()

In [ ]:
census.drop(columns= ['Unnamed: 0'])
census.head(2)

In [ ]:
census.rename(columns= {'Unnamed: 1': 'Place'})
census.head(2)

In [ ]:
census[[2010,2011, 2012, 2013, 2014, 2015]].apply(lambda x: x.std(), axis=1)

In [ ]:
1e7

In [ ]:
census[[2010,2011, 2012, 2013, 2014, 2015]].applymap(lambda x: x/1e3, )

In [ ]:
## Splitting cities and states 

In [ ]:
census['city'] = census['Unnamed: 1'].map(lambda x: x.split(',')[0][:-5])
census['state'] = census['Unnamed: 1'].map(lambda x: x.split(',')[1].strip(' '))

In [ ]:
## creating the abbrevated state names

In [ ]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


In [ ]:
states = {values: key for key, values in states.items()}

In [ ]:
census['abbr'] = census.state.map(lambda state: states[state])

In [ ]:
census.head(2)

[More on the differences between `apply`,`map` and `applymap`](https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas)

## Merging Datasets

This Part of the notebook is adopted from:

[PythonDataScienceHandbook - Merge -Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)

In [ ]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

In [ ]:


df3 = pd.merge(df1, df2)
display('df1', 'df2','pd.merge(df1, df2)')



In [ ]:


df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')



In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

For more details please check this excellent notebook:

[Python Data Science Handbook - Merge-Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)

And the `pandas` documentation

[Pandas - Merging](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

In [ ]:
df_merged = pd.merge(census[['city', 'state', 2016]], df, on = 'city', how = 'inner')

In [ ]:
df_merged.head()

## Groupby

In [ ]:
grouped = df_merged.groupby(by = 'city')
grouped.get_group('New York').race.value_counts()

In [ ]:
grouped[2016].mean()

In [ ]:
grouped.aggregate(['min', np.median, max])['age']

In [ ]:
my_func = lambda x: x.max() - x.min()

In [ ]:
grouped.aggregate({'date': 'max', 'age': 'min', 2016: my_func})

In [ ]:
grouped = df_merged.groupby(by = ['city', 'state_x'])


In [ ]:
grouped.get_group(('New York', 'New York')).race.value_counts()

In [ ]:
grouped[2016].mean()

In [ ]:
(grouped['city'].count()/grouped[2016].mean()).sort_values(ascending =False)

In [ ]:
grouped.get_group('Toledo')

## Resources

[More on encodings](https://realpython.com/python-encodings-guide/)